In [1]:
def triangle(x, f = 0.1):
    return np.acos((1.0 - f) * np.cos(x)) - np.pi * 0.5

In [2]:
# import cmath
import numpy as np

class Phasor:
    def __init__(self, func = triangle):
        self.mem = [0.0, 0.0]
        self.pi = np.pi
        self.tao = np.pi*2
        self.f = func
        
    def tick(self, phase, pwm, sl = 0.5, sr = 0.5, sat = 0.6):
        # First half oscillator
        oa = self.f(phase + self.mem[0], sat)
        self.mem[0] = sl * (oa + self.mem[0])  # anti-hunting filter

        # Phase offset for PWM [-0.49 ... 0.49]
        pw = pwm * self.tao - self.pi

        # Second half oscillator
        ob = self.f(phase + self.mem[1] + pw, sat)
        self.mem[1] = sr * (ob + self.mem[1])  # anti-hunting filter

        return oa - ob


        

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from IPython.display import Audio

def plot_tomisawa(pwm=0.3, ahl = 0.5, ahr = 0.5, sat = 0.6):
    sample_rate = 44100
    freq = 110
    duration = 0.03
    samples = int(sample_rate * duration)
    theta = 2 * np.pi * freq / sample_rate
    phase = -np.pi

    osc = Phasor()
    wave = np.zeros(samples, dtype=np.float32)

    for i in range(samples):
        phase += theta
        if phase > np.pi:
            phase -= 2 * np.pi
        wave[i] = osc.tick(phase, pwm, ahl, ahr, sat)

    wave *= 0.9 / np.max(np.abs(wave))  # normalize

    # FFT
    fft = np.fft.fft(wave)
    freqs = np.fft.fftfreq(len(fft), 1/sample_rate)
    spectrum = 20 * np.log10(np.abs(fft[:samples//4]) + 1e-6)

    # Plot
    fig, axs = plt.subplots(2, 1, figsize=(10, 5))
    t = np.linspace(0, duration, samples)

    axs[0].plot(t, wave)
    axs[0].set_title(f"Tomisawa PWM Waveform (PWM={pwm:.2f})")
    axs[0].set_xlabel("Time (s)")
    axs[0].set_ylabel("Amplitude")
    axs[0].grid(True)

    axs[1].plot(freqs[:samples//4], spectrum)
    axs[1].set_title("FFT Spectrum")
    axs[1].set_xlabel("Frequency (Hz)")
    axs[1].set_ylabel("Magnitude (dB)")
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()

interact(
    plot_tomisawa,
    pwm=FloatSlider(value=0.0, min=-0.49, max=0.49, step=0.01, description="PWM"),
    ahl=FloatSlider(value=0.5, min=-0.9, max=0.9, step=0.001, description="AHL"),
    ahr=FloatSlider(value=0.5, min=-0.9, max=0.9, step=0.001, description="AHR"),
    sat=FloatSlider(value=0.5, min=0.001, max=0.6, step=0.001, description="Saturation")
)


interactive(children=(FloatSlider(value=0.0, description='PWM', max=0.49, min=-0.49, step=0.01), FloatSlider(v…

<function __main__.plot_tomisawa(pwm=0.3, ahl=0.5, ahr=0.5, sat=0.6)>